# Miami Housing · Projeto de Análise e Modelagem

## Objetivo
Investigar os determinantes do preço de venda de imóveis residenciais em Miami (2016), estruturando um fluxo incremental que contempla: (i) descoberta e inspeção do conjunto de dados; (ii) limpeza e padronização (*tidy*); (iii) análise exploratória com gramática de gráficos; e (iv) preparação de artefatos para etapas posteriores de modelagem preditiva e inferencial.

## Equipe
- João Pedro Queiroz Viana  
- João Gabriel Faus Faraco


## Importações e configurações

In [66]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR = Path("data")
print(DATA_DIR)

data


## Obtendo os dados

In [67]:
import pandas as pd


def load_housing_data(data_dir: Path) -> pd.DataFrame:
    '''Loads the Miami Housing dataset.

    Loads the Miami Housing dataset from the specified directory.

    Args:
        data_dir: The directory from which the dataset will be loaded.

    Returns:
        A pandas DataFrame containing the California Housing Prices dataset.
    '''
    csv_path = data_dir / 'miami-housing.csv'
    df = pd.read_csv(csv_path)
    return df

In [68]:
data = load_housing_data(DATA_DIR)

print(f'O dataset tem {data.shape[0]} linhas e {data.shape[1]} colunas.')
print('As colunas são:')
for column_name in data.columns:
    print(f'- "{column_name}"')

O dataset tem 13932 linhas e 17 colunas.
As colunas são:
- "LATITUDE"
- "LONGITUDE"
- "PARCELNO"
- "SALE_PRC"
- "LND_SQFOOT"
- "TOT_LVG_AREA"
- "SPEC_FEAT_VAL"
- "RAIL_DIST"
- "OCEAN_DIST"
- "WATER_DIST"
- "CNTR_DIST"
- "SUBCNTR_DI"
- "HWY_DIST"
- "age"
- "avno60plus"
- "month_sold"
- "structure_quality"


## Entendimento do Negócio

### O que os dados representam

Este dataset (arquivo `miami-housing.csv`, 13.932 linhas × 17 colunas) contém **registros de transações imobiliárias residenciais** na região de Miami (EUA), com:
- **Preço de venda** do imóvel,
- **Características físicas** do lote e da área construída,
- **Localização geográfica** (latitude/longitude),
- **Qualidade estrutural** do imóvel,
- **Proximidades e acessibilidade** (distâncias até mar/água, ferrovia, rodovias e centros urbanos),
- **Atributos contextuais** (ex.: mês da venda; indicador binário relacionado a ruído/ambiente — ver nota abaixo).

> **Objetivo típico de negócio:** apoiar análises de **precificação**, avaliação de **fatores que influenciam valor** (amenidades e acessibilidade), **segmentação** de portfólio e **modelagem preditiva** do preço de venda.

> **Observação importante:** a coluna `avno60plus` é um indicador binário (0/1). Pelo padrão de datasets imobiliários, é **provavelmente** um marcador de **exposição a ruído acima de 60 dB** (ex.: ruído aeronáutico “60+ dB”). Recomenda-se **confirmar no dicionário de dados oficial** do projeto.

---

### Descrição das Colunas

| Coluna              | Tipo      | Tipo de Variável (Natureza) | Descrição de Negócio                                                                  | Unidade / Domínio                        | Exemplos / Faixa (min–max)              |
| ------------------- | --------- | --------------------------- | ------------------------------------------------------------------------------------- | ---------------------------------------- | --------------------------------------- |
| `LATITUDE`          | float     | Contínua                    | Latitude geográfica do imóvel (para georreferenciamento e mapas).                     | Graus decimais                           | 25.4343 – 25.9744                       |
| `LONGITUDE`         | float     | Contínua                    | Longitude geográfica do imóvel.                                                       | Graus decimais                           | −80.5422 – −80.1198                     |
| `PARCELNO`          | int       | Categórica (ID)             | Identificador cadastral do lote/parcela (parcel number).                              | ID numérico                              | ~1.02e11 – 3.66e12                      |
| `SALE_PRC`          | float     | Contínua                    | **Preço de venda** do imóvel (variável-alvo comum em modelos).                        | USD                                      | **72.000 – 2.650.000**                  |
| `LND_SQFOOT`        | int       | Contínua                    | **Área do terreno (lote)**.                                                           | Pés quadrados                            | 1.248 – 57.064                          |
| `TOT_LVG_AREA`      | int       | Contínua                    | **Área construída habitável** (living area).                                          | Pés quadrados                            | 854 – 6.287                             |
| `SPEC_FEAT_VAL`     | int       | Contínua                    | Valor agregado por **características especiais** (ex.: piscina, lareira, deck).       | USD (valor estimado de amenidades)       | 0 – 175.020                             |
| `RAIL_DIST`         | float     | Contínua                    | Distância à **ferrovia** (proxy de acessibilidade/ruído).                             | Metros (aprox.)                          | 10,5 – 29.621,5                         |
| `OCEAN_DIST`        | float     | Contínua                    | Distância ao **oceano** (amenidade costeira).                                         | Metros (aprox.)                          | 236,1 – 75.744,9                        |
| `WATER_DIST`        | float     | Contínua                    | Distância ao **corpo d’água** mais próximo (lagos/canais, exceto oceano).             | Metros (aprox.)                          | 0 – 50.399,8                            |
| `CNTR_DIST`         | float     | Contínua                    | Distância ao **CBD** (centro principal da cidade).                                    | Metros (aprox.)                          | 3.825,6 – 159.976,5                     |
| `SUBCNTR_DI`        | float     | Contínua                    | Distância ao **subcentro** urbano mais próximo.                                       | Metros (aprox.)                          | 1.462,8 – 110.553,8                     |
| `HWY_DIST`          | float     | Contínua                    | Distância à **rodovia** mais próxima.                                                 | Metros (aprox.)                          | 90,2 – 48.167,3                         |
| `age`               | int       | Contínua                    | **Idade do imóvel** na data da venda.                                                 | Anos                                     | 0 – 96                                  |
| `avno60plus`        | int (0/1) | Categórica (Binária)        | **Indicador binário** relacionado ao ambiente; **provável** exposição a ruído ≥60 dB. | 0 = não exposto; 1 = exposto (confirmar) | 0 ou 1 (média ~0,015)                   |
| `month_sold`        | int       | Categórica (Ordinal)        | **Mês da venda** (sazonalidade de mercado).                                           | 1–12                                     | 1 – 12 (distribuição em todos os meses) |
| `structure_quality` | int       | Categórica (Ordinal)        | **Qualidade construtiva** em escala ordinal.                                          | Escala 1–5 (1=baixa, 5=alta)             | 1–5 (moda ~4)                           |

### Leituras rápidas de negócio

- **Preço de venda (`SALE_PRC`)** tende a ser explicado por: área construída, área do lote, qualidade, idade, amenidades (`SPEC_FEAT_VAL`) e **localização** (distâncias a mar/água/centros/rodovias).
- **Localização importa**: menor `OCEAN_DIST` e `WATER_DIST` (maior proximidade à água) geralmente aumentam valor; já menor `RAIL_DIST`/`HWY_DIST` pode ter efeitos ambíguos (acessibilidade vs. ruído).
- **Sazonalidade**: `month_sold` ajuda a capturar ciclos de demanda/oferta no ano.
- **Qualidade e idade**: `structure_quality` e `age` capturam condição do imóvel; imóveis mais novos e com melhor padrão costumam precificar acima da média.

## Relações a serem observadas

### Características físicas e estruturais:
1. Esta etapa busca investigar quais atributos dos imóveis explicam a variação nos preços de venda `SALE_PRC`. Serão analisadas variáveis como área construída `TOT_LVG_AREA`, área do lote `LND_SQFOOT`, qualidade estrutural `structure_quality` e idade `age`. O objetivo é verificar se há correlação positiva entre tamanho e preço, identificar possíveis retornos decrescentes em terrenos muito amplos, avaliar se maiores níveis de qualidade estão associados a valores mais altos e investigar se imóveis mais antigos tendem a apresentar menor valorização.

### Fatores geográficos e localização:
2. Nesta análise, o foco está em avaliar o impacto da localização sobre o valor dos imóveis, compreendendo como fatores geográficos influenciam o preço de venda `SALE_PRC`. Serão examinadas as distâncias em relação a pontos de interesse e fontes de ruído, como a proximidade ao oceano `OCEAN_DIST` e a corpos d’água `WATER_DIST`, que podem indicar valorização de áreas costeiras. Também será investigado o efeito de centralidade, observando se imóveis mais próximos ao centro urbano `CNTR_DIST` e a subcentros `SUBCNTR_DI` apresentam preços mais elevados. Além disso, será analisada a influência da distância a rodovias `HWY_DIST`, que pode ter efeitos ambíguos — associando-se tanto à acessibilidade quanto ao ruído e à poluição. Por fim, pretende-se explorar se a proximidade à ferrovia `RAIL_DIST` e a exposição a ruído (avno60plus) impactam negativamente a atratividade e o valor dos imóveis.

### Aspectos contextuais e sazonais:
3. O objetivo desta etapa é analisar padrões sazonais e fatores contextuais que influenciam o preço de venda `SALE_PRC`. Serão avaliados o mês da venda `month_sold`, para identificar possíveis variações sazonais, e o valor das amenidades `SPEC_FEAT_VAL`, como piscinas ou lareiras, que podem agregar valor ao imóvel. Além disso, será considerada a variável `avno60plus`, que indica exposição a ruído, a fim de verificar se ambientes mais ruidosos tendem a apresentar menor valorização.

## Análise exploratória

### Análise global

In [69]:
data.head()

,LATITUDE,LONGITUDE,PARCELNO,SALE_PRC,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality
0,25.891031,-80.160561,622280070620,440000.0,9375,1753,0,2815.9,12811.4,347.6,42815.3,37742.2,15954.9,67,0,8,4
1,25.891324,-80.153968,622280100460,349000.0,9375,1715,0,4359.1,10648.4,337.8,43504.9,37340.5,18125.0,63,0,9,4
2,25.891334,-80.153740,622280100470,800000.0,9375,2276,49206,4412.9,10574.1,297.1,43530.4,37328.7,18200.5,61,0,2,4
3,25.891765,-80.152657,622280100530,988000.0,12450,2058,10033,4585.0,10156.5,0.0,43797.5,37423.2,18514.4,63,0,9,4
4,25.891825,-80.154639,622280100200,755000.0,12800,1684,16681,4063.4,10836.8,326.6,43599.7,37550.8,17903.4,42,0,7,4


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13932 entries, 0 to 13931
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   LATITUDE           13932 non-null  float64
 1   LONGITUDE          13932 non-null  float64
 2   PARCELNO           13932 non-null  int64  
 3   SALE_PRC           13932 non-null  float64
 4   LND_SQFOOT         13932 non-null  int64  
 5   TOT_LVG_AREA       13932 non-null  int64  
 6   SPEC_FEAT_VAL      13932 non-null  int64  
 7   RAIL_DIST          13932 non-null  float64
 8   OCEAN_DIST         13932 non-null  float64
 9   WATER_DIST         13932 non-null  float64
 10  CNTR_DIST          13932 non-null  float64
 11  SUBCNTR_DI         13932 non-null  float64
 12  HWY_DIST           13932 non-null  float64
 13  age                13932 non-null  int64  
 14  avno60plus         13932 non-null  int64  
 15  month_sold         13932 non-null  int64  
 16  structure_quality  139

In [71]:
data_no_dups = data.drop_duplicates()

In [72]:
data_no_dups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13932 entries, 0 to 13931
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   LATITUDE           13932 non-null  float64
 1   LONGITUDE          13932 non-null  float64
 2   PARCELNO           13932 non-null  int64  
 3   SALE_PRC           13932 non-null  float64
 4   LND_SQFOOT         13932 non-null  int64  
 5   TOT_LVG_AREA       13932 non-null  int64  
 6   SPEC_FEAT_VAL      13932 non-null  int64  
 7   RAIL_DIST          13932 non-null  float64
 8   OCEAN_DIST         13932 non-null  float64
 9   WATER_DIST         13932 non-null  float64
 10  CNTR_DIST          13932 non-null  float64
 11  SUBCNTR_DI         13932 non-null  float64
 12  HWY_DIST           13932 non-null  float64
 13  age                13932 non-null  int64  
 14  avno60plus         13932 non-null  int64  
 15  month_sold         13932 non-null  int64  
 16  structure_quality  139

In [73]:
summary = pd.DataFrame({
    "coluna": data.columns,
    "tipo": [str(t) for t in data.dtypes.values],
    "valores_nulos": data.isna().sum().values,
    "valores_nao_nulos": data.notna().sum().values,
    "valores_unicos": data.nunique(dropna=True).values
})


summary


,coluna,tipo,valores_nulos,valores_nao_nulos,valores_unicos
0,LATITUDE,float64,0,13932,13776
1,LONGITUDE,float64,0,13932,13776
2,PARCELNO,int64,0,13932,13776
3,SALE_PRC,float64,0,13932,2111
4,LND_SQFOOT,int64,0,13932,4696
5,TOT_LVG_AREA,int64,0,13932,2978
6,SPEC_FEAT_VAL,int64,0,13932,7583
7,RAIL_DIST,float64,0,13932,13235
8,OCEAN_DIST,float64,0,13932,13617
9,WATER_DIST,float64,0,13932,13218


### Análise preliminar das features

In [74]:
categorical_features = ['month_sold', 'structure_quality', 'avno60plus', 'PARCELNO']

numeric_data = data.select_dtypes(include='number').drop(columns=categorical_features, errors='ignore')

In [75]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
LATITUDE,13932.0,2.572881e+01,1.406333e-01,2.543433e+01,2.562006e+01,2.573181e+01,2.585227e+01,2.597438e+01
LONGITUDE,13932.0,-8.032748e+01,8.919907e-02,-8.054217e+01,-8.040328e+01,-8.033891e+01,-8.025802e+01,-8.011975e+01
PARCELNO,13932.0,2.356496e+12,1.199290e+12,1.020008e+11,1.079160e+12,3.040300e+12,3.060170e+12,3.660170e+12
SALE_PRC,13932.0,3.999419e+05,3.172147e+05,7.200000e+04,2.350000e+05,3.100000e+05,4.280000e+05,2.650000e+06
LND_SQFOOT,13932.0,8.620880e+03,6.070089e+03,1.248000e+03,5.400000e+03,7.500000e+03,9.126250e+03,5.706400e+04
TOT_LVG_AREA,13932.0,2.058045e+03,8.135385e+02,8.540000e+02,1.470000e+03,1.877500e+03,2.471000e+03,6.287000e+03
SPEC_FEAT_VAL,13932.0,9.562493e+03,1.389097e+04,0.000000e+00,8.100000e+02,2.765500e+03,1.235225e+04,1.750200e+05
RAIL_DIST,13932.0,8.348549e+03,6.178027e+03,1.050000e+01,3.299450e+03,7.106300e+03,1.210260e+04,2.962150e+04
OCEAN_DIST,13932.0,3.169099e+04,1.759508e+04,2.361000e+02,1.807935e+04,2.854175e+04,4.431065e+04,7.574490e+04
WATER_DIST,13932.0,1.196029e+04,1.193299e+04,0.000000e+00,2.675850e+03,6.922600e+03,1.920000e+04,5.039980e+04


In [76]:
def compute_coefficient_of_variation(data: pd.DataFrame) -> pd.Series:
    """
    Computes the Coefficient of Variation (CV) for each numerical column 
    in a DataFrame, excluding categorical or discrete features.

    Args:
        data: A pandas DataFrame.

    Returns:
        A pandas Series containing the Coefficient of Variation for each
        selected column.
    """
    categorical_features = ['month_sold', 'structure_quality', 'avno60plus', 'PARCELNO']
    
    numeric_cols = [
        col for col in data.select_dtypes(include='number').columns
        if col not in categorical_features
    ]
    
    stats = data[numeric_cols].describe().transpose()
    
    cv = stats['std'] / stats['mean']
    cv.rename('Coefficient of Variation', inplace=True)
    
    return cv

print(compute_coefficient_of_variation(data).round(2).to_markdown())

|               |   Coefficient of Variation |
|:--------------|---------------------------:|
| LATITUDE      |                       0.01 |
| LONGITUDE     |                      -0    |
| SALE_PRC      |                       0.79 |
| LND_SQFOOT    |                       0.7  |
| TOT_LVG_AREA  |                       0.4  |
| SPEC_FEAT_VAL |                       1.45 |
| RAIL_DIST     |                       0.74 |
| OCEAN_DIST    |                       0.56 |
| WATER_DIST    |                       1    |
| CNTR_DIST     |                       0.47 |
| SUBCNTR_DI    |                       0.54 |
| HWY_DIST      |                       0.79 |
| age           |                       0.69 |
